In [2]:
# 드래프트 연도 기준 프로대뷔 3년차 이하 선수들 리스트 크롤링

from bs4 import BeautifulSoup
import pandas as pd
import datetime
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time
from selenium.webdriver.common.by import By
from itertools import count
import time
from tqdm import tqdm_notebook
import pdb
import re

driver = webdriver.Chrome(r'C:/Users/Yun/Desktop/Jupyter/chromedriver.exe')
driver.get('https://kbl.or.kr/story/draft')
draft_year_select = driver.find_element(By.XPATH, '//*[@id="container"]/div/ul/li[2]/select') #결과 버튼

year_list = draft_year_select.find_elements(By.TAG_NAME, "option")
draft_year = ['2021 KBL 신인선수 드래프트 결과', '2020 KBL 국내신인선수 드래프트 결과']
#2020년도 지명 선수부터를 젊은 선수로 선정하였고 2022년도 지명 선수는 기록 양이 너무 적어 제외하였습니다.
player_df = pd.DataFrame(columns=("Player", "Posiiton"))
player = []
position = []
for draft in draft_year:
    #드래프트 년도 박스 선택
    select = Select(driver.find_element(By.XPATH, '//*[@id="container"]/div/ul/li[2]/select'))
    select.select_by_value(draft)
    #드래프트 선발 선수 이름 크롤링
    try:
        name_first_round = driver.find_element(By.CSS_SELECTOR, '#container > div > div.con-box > div.scroll-table > table > tbody:nth-child(2) > tr:nth-child(2)').text
        name_second_round = driver.find_element(By.CSS_SELECTOR, '#container > div > div.con-box > div.scroll-table > table > tbody:nth-child(4) > tr:nth-child(2)').text
        name_third_round = driver.find_element(By.CSS_SELECTOR, '#container > div > div.con-box > div.scroll-table > table > tbody:nth-child(6) > tr:nth-child(2)').text
        name_fourth_round = driver.find_element(By.CSS_SELECTOR, '#container > div > div.con-box > div.scroll-table > table > tbody:nth-child(8) > tr:nth-child(2)').text
    except Exception as e:
        print(" ")
        
    try:
        position_first_round = driver.find_element(By.CSS_SELECTOR, '#container > div > div.con-box > div.scroll-table > table > tbody:nth-child(2) > tr:nth-child(4)').text
        position_second_round = driver.find_element(By.CSS_SELECTOR, '#container > div > div.con-box > div.scroll-table > table > tbody:nth-child(4) > tr:nth-child(4)').text
        position_third_round = driver.find_element(By.CSS_SELECTOR, '#container > div > div.con-box > div.scroll-table > table > tbody:nth-child(6) > tr:nth-child(4)').text
        position_fourth_round = driver.find_element(By.CSS_SELECTOR, '#container > div > div.con-box > div.scroll-table > table > tbody:nth-child(8) > tr:nth-child(4)').text
    except Exception as e:
        print(" ")
    #드래프트에서 팀이 지명하지 않은 경우에 '지명권포기'를 리스트에서 제거 및 빈칸 제거
    position_1round = position_first_round.split(" ")
    position_2round = position_second_round.split(" ")
    position_3round = position_third_round.split(" ")
    position_4round = position_fourth_round.split(" ")
    
    name_1round = name_first_round.split(" ")
    name_2round = name_second_round.split(" ")
    name_3round = name_third_round.split(" ")
    name_4round = name_fourth_round.split(" ")
    
    player.append(name_1round)
    player.append(name_2round)
    player.append(name_3round)
    player.append(name_4round)
    
    position.append(position_1round)
    position.append(position_2round)
    position.append(position_3round)
    position.append(position_4round)
    
    time.sleep(1)

    
position_result = sum(position, [])
player_list = sum(player, [])
remove_set = {'지명권포기', '1R', '2R', '3R', '4R'}
player_result = [i for i in player_list if i not in remove_set]


player_df = pd.DataFrame(player_result, columns = ['player'])
player_df['position'] = position_result

C:\Users\Yun\AppData\Local\Temp\ipykernel_10480\388117143.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'C:/Users/Yun/Desktop/Jupyter/chromedriver.exe')


In [3]:
#크롤링된 대뷔 3년 이하 선수들 기록 크롤링

G_rookie=[]    #게임수
PPG_rookie=[]    #경기당 평균 득점
FG_rookie=[]    #필드골 성공
P3_rookie=[]    #3점슛 성공
FT_rookie=[]    #자유투 성공
RPG_rookie=[]    #경기당 리바운드
APG_rookie=[]    #경기당 어시스트
SPG_rookie=[]    #경기당 스틸
BPG_rookie=[]    #경기당 블록
TO_rookie=[]    #턴오버
PF_rookie=[]    #파울
EFF_rookie=[]    #효율성수치


for i in range(0,len(player_result)):
    # 선수이름 검색
    time.sleep(1)
    driver.get('https://kbl.or.kr/player-staff/player')
    search_space = driver.find_element(By.CLASS_NAME, "cs")
    search_space.clear()
    search_space.send_keys(player_result[i])
    driver.find_element(By.XPATH, '//*[@id="container"]/div/div[3]/div/button/i').click()
    firstname = driver.find_element(By.CSS_SELECTOR, '#container > div > div.con-search > ul').text
    time.sleep(1)
    #hangul-ㄱ > ul > li:nth-child(1) > div.img
    //*[@id="hangul-ㄱ"]/ul/li[1]/div[1]
    //*[@id="hangul-ㄱ"]/ul/li[1]
    //*[@id="hangul-ㄱ"]/ul/li[1]
    //*[@id="hangul-ㄱ"]/ul/li[1]/div[1]
    #검색되지 않는다면
    if firstname == '':
        g_rookie = 0
        ppg_rookie = 0
        fg_rookie = 0
        p3_rookie = 0
        ft_rookie = 0
        rpg_rookie = 0
        apg_rookie = 0
        spg_rookie = 0
        bpg_rookie = 0
        to_rookie = 0
        pf_rookie = 0
        eff_rookie = 0
        
    else:
        player_detail = f'//*[@id="hangul-{firstname}"]/ul/li/div[1]'
        driver.find_element(By.XPATH, player_detail).click()
        time.sleep(3)
        league_name=driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(5) > div.con-tit > h4').text
        if league_name=="정규경기 시즌별 기록":
            try:#tbody > tr.average > td.game
                g_rookie = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(5) > div.season-record-table01 > table > tbody > tr.average > td.game').text
                ppg_rookie = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(5) > div.season-record-table01 > table > tbody > tr.average > td.ppg').text
                fg_rookie = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(5) > div.season-record-table01 > table > tbody > tr.average > td.fg').text
                p3_rookie = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(5) > div.season-record-table01 > table > tbody > tr.average > td.p3').text
                ft_rookie = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(5) > div.season-record-table01 > table > tbody > tr.average > td.ft').text
                rpg_rookie = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(5) > div.season-record-table01 > table > tbody > tr.average > td.rpg').text
                apg_rookie = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(5) > div.season-record-table01 > table > tbody > tr.average > td.apg').text
                spg_rookie = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(5) > div.season-record-table01 > table > tbody > tr.average > td.spg').text
                bpg_rookie = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(5) > div.season-record-table01 > table > tbody > tr.average > td.bpg').text
                to_rookie = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(5) > div.season-record-table01 > table > tbody > tr.average > td.to').text
                pf_rookie = driver.find_elemen(By.CSS_SELECTOR, '#container > div > div:nth-child(5) > div.season-record-table01 > table > tbody > tr.average > td.pf').text
                eff_rookie = float(ppg_rookie)+float(rpg_rookie)+float(apg_rookie)+float(bpg_rookie)-float(to_rookie)
            except Exception as e:
                g_rookie = 0
                ppg_rookie = 0
                fg_rookie = 0
                p3_rookie = 0
                ft_rookie = 0
                rpg_rookie = 0
                apg_rookie = 0
                spg_rookie = 0
                bpg_rookie = 0
                to_rookie = 0
                pf_rookie = 0
                eff_rookie = 0
        else:
            try:
                g_rookie = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(4) > div.season-record-table01 > table > tbody > tr.average > td.game').text
                ppg_rookie = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(4) > div.season-record-table01 > table > tbody > tr.average > td.ppg').text
                fg_rookie = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(4) > div.season-record-table01 > table > tbody > tr.average > td.fg').text
                p3_rookie = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(4) > div.season-record-table01 > table > tbody > tr.average > td.p3').text
                ft_rookie = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(4) > div.season-record-table01 > table > tbody > tr.average > td.ft').text
                rpg_rookie = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(4) > div.season-record-table01 > table > tbody > tr.average > td.rpg').text
                apg_rookie = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(4) > div.season-record-table01 > table > tbody > tr.average > td.apg').text
                spg_rookie = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(4) > div.season-record-table01 > table > tbody > tr.average > td.spg').text
                bpg_rookie = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(4) > div.season-record-table01 > table > tbody > tr.average > td.bpg').text
                to_rookie = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(4) > div.season-record-table01 > table > tbody > tr.average > td.to').text
                pf_rookie = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(4) > div.season-record-table01 > table > tbody > tr.average > td.pf').text
                eff_rookie = float(ppg_rookie)+float(rpg_rookie)+float(apg_rookie)+float(bpg_rookie)-float(to_rookie)
                
            except Exception as e:
                g_rookie = 0
                ppg_rookie = 0
                fg_rookie = 0
                p3_rookie = 0
                ft_rookie = 0
                rpg_rookie = 0
                apg_rookie = 0
                spg_rookie = 0
                bpg_rookie = 0
                to_rookie = 0
                pf_rookie = 0
                eff_rookie = 0

            
        
    G_rookie.append(g_rookie)
    PPG_rookie.append(ppg_rookie)
    FG_rookie.append(fg_rookie)
    P3_rookie.append(p3_rookie)
    FT_rookie.append(ft_rookie)
    RPG_rookie.append(rpg_rookie)
    APG_rookie.append(apg_rookie)
    SPG_rookie.append(spg_rookie)
    BPG_rookie.append(bpg_rookie)
    TO_rookie.append(to_rookie)
    PF_rookie.append(pf_rookie)
    EFF_rookie.append(eff_rookie)

    
player_df['게임수'] = G_rookie
player_df['평균득점'] = PPG_rookie
player_df['필드골'] = FG_rookie
player_df['3점'] = P3_rookie
player_df['자유투'] = FT_rookie
player_df['리바운드'] = RPG_rookie
player_df['어시스트'] = APG_rookie
player_df['스틸'] = SPG_rookie
player_df['블록'] = BPG_rookie
player_df['턴오버'] = TO_rookie
player_df['파울'] = PF_rookie
player_df['효율성'] = EFF_rookie

player_df.to_csv("rookie_player.csv", index = False)

In [4]:
#전체선수 기록 크롤링

Name=[]
G=[]    #게임수
PPG=[]    #경기당 평균 득점
FG=[]    #필드골 성공
P3=[]    #3점슛 성공
FT=[]    #자유투 성공
RPG=[]    #경기당 리바운드
APG=[]    #경기당 어시스트
SPG=[]    #경기당 스틸
BPG=[]    #경기당 블록
TO=[]    #턴오버
PF=[]    #파울
EFF=[]    #효율성수치
locate=[]

driver.get('https://kbl.or.kr/player-staff/player')
time.sleep(1)
Root_class=driver.find_elements(By.CLASS_NAME, "player-list-wrap")
for Root in Root_class:
    li_tag=Root.find_elements(By.TAG_NAME, "li")
    for code in li_tag:
        locate.append(code.get_attribute("data-pcode"))


for location in locate:
    time.sleep(1)
    driver.get(f'https://kbl.or.kr/player-staff/player/{location}')
    time.sleep(3)
    try:
        league_name=driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(5) > div.con-tit > h4').text
        if league_name=="정규경기 시즌별 기록":
            try:
                name = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(2) > div.player-view-wrap > div.txt > div > div.name > h6').text
                g = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(5) > div.season-record-table01 > table > tbody > tr.average > td.game').text
                ppg = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(5) > div.season-record-table01 > table > tbody > tr.average > td.ppg').text
                fg = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(5) > div.season-record-table01 > table > tbody > tr.average > td.fg').text
                p3 = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(5) > div.season-record-table01 > table > tbody > tr.average > td.p3').text
                ft = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(5) > div.season-record-table01 > table > tbody > tr.average > td.ft').text
                rpg = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(5) > div.season-record-table01 > table > tbody > tr.average > td.rpg').text
                apg = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(5) > div.season-record-table01 > table > tbody > tr.average > td.apg').text
                spg = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(5) > div.season-record-table01 > table > tbody > tr.average > td.spg').text
                bpg = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(5) > div.season-record-table01 > table > tbody > tr.average > td.bpg').text
                to = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(5) > div.season-record-table01 > table > tbody > tr.average > td.to').text
                pf = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(5) > div.season-record-table01 > table > tbody > tr.average > td.pf').text
                eff = float(ppg)+float(rpg)+float(apg)+float(bpg)-float(to)
            except Exception as e:
                name = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(2) > div.player-view-wrap > div.txt > div > div.name > h6').text
                g = 0
                ppg = 0
                fg = 0
                p3 = 0
                ft = 0
                rpg = 0
                apg = 0
                spg = 0
                bpg = 0
                to = 0
                pf = 0
                eff = 0
        else:
            try:
                name = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(2) > div.player-view-wrap > div.txt > div > div.name > h6').text
                g = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(4) > div.season-record-table01 > table > tbody > tr.average > td.game').text
                ppg = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(4) > div.season-record-table01 > table > tbody > tr.average > td.ppg').text
                fg = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(4) > div.season-record-table01 > table > tbody > tr.average > td.fg').text
                p3 = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(4) > div.season-record-table01 > table > tbody > tr.average > td.p3').text
                ft = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(4) > div.season-record-table01 > table > tbody > tr.average > td.ft').text
                rpg = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(4) > div.season-record-table01 > table > tbody > tr.average > td.rpg').text
                apg = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(4) > div.season-record-table01 > table > tbody > tr.average > td.apg').text
                spg = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(4) > div.season-record-table01 > table > tbody > tr.average > td.spg').text
                bpg = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(4) > div.season-record-table01 > table > tbody > tr.average > td.bpg').text
                to = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(4) > div.season-record-table01 > table > tbody > tr.average > td.to').text
                pf = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(4) > div.season-record-table01 > table > tbody > tr.average > td.pf').text
                eff = float(ppg)+float(rpg)+float(apg)+float(bpg)-float(to)

            except Exception as e:
                name = driver.find_element(By.CSS_SELECTOR, '#container > div > div:nth-child(2) > div.player-view-wrap > div.txt > div > div.name > h6').text
                g = 0
                ppg = 0
                fg = 0
                p3 = 0
                ft = 0
                rpg = 0
                apg = 0
                spg = 0
                bpg = 0
                to = 0
                pf = 0
                eff = 0
    except Exception as e:
        pass
    
    Name.append(name)
    G.append(g)
    PPG.append(ppg)
    FG.append(fg)
    P3.append(p3)
    FT.append(ft)
    RPG.append(rpg)
    APG.append(apg)
    SPG.append(spg)
    BPG.append(bpg)
    TO.append(to)
    PF.append(pf)
    EFF.append(eff)
     
all_player_df = pd.DataFrame(Name, columns = ['player'])
all_player_df['게임수'] = G
all_player_df['평균득점'] = PPG
all_player_df['필드골'] = FG
all_player_df['3점'] = P3
all_player_df['자유투'] = FT
all_player_df['리바운드'] = RPG
all_player_df['어시스트'] = APG
all_player_df['스틸'] = SPG
all_player_df['블록'] = BPG
all_player_df['턴오버'] = TO
all_player_df['파울'] = PF
all_player_df['효율성'] = EFF

all_player_df.to_csv("all_player.csv", index = False)

In [11]:
#나무위키에 시즌마다 웹페이지 구성이 조금씩 달라 각 년도마다 다르게 크롤링하였습니다.
# 2022시즌 best선수 이름 크롤링


from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

headers = {
    'User-Agent': 'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.3; WOW64; Trident/7.0)'
}

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
season_list = ['2021-22', '2020-21', '2019-20', '2018-19', '2017-18', '2016-17', '2015-16', '2014-15']
best5_list_22 = []
best5_position_22 = []
driver.get(f'https://namu.wiki/w/%ED%8B%80:KBL%20%EB%B2%A0%EC%8A%A4%ED%8A%B8%205({season_list[0]}%20%EC%8B%9C%EC%A6%8C)')

driver.find_element(By.XPATH, '//*[@id="lYnqWtHhT"]/div[2]/div/div/div/div/div[1]/div[6]/div/div/div/div/div/div[6]/div/div/div/div/div/div/div/div/div[15]/div[1]/div/div[1]/table/tbody/tr[2]/td/div/dl/dt').click()
time.sleep(1)

for i in range(1,6):
    name = driver.find_element(By.XPATH, f'//*[@id="lYnqWtHhT"]/div[2]/div/div/div/div/div[1]/div[6]/div/div/div/div/div/div[6]/div/div/div/div/div/div/div/div/div[15]/div[1]/div/div[1]/table/tbody/tr[2]/td/div/dl/dd/div/div/table/tbody/tr[2]/td[{i}]/div/a').text
    position = driver.find_element(By.XPATH, f'//*[@id="lYnqWtHhT"]/div[2]/div/div/div/div/div[1]/div[6]/div/div/div/div/div/div[6]/div/div/div/div/div/div/div/div/div[15]/div[1]/div/div[1]/table/tbody/tr[2]/td/div/dl/dd/div/div/table/tbody/tr[2]/td[{i}]/div/sub/a[2]').text
    best5_list_22.append(name)
    best5_position_22.append(position)

driver.close()

In [12]:
# 2021시즌 best선수 이름 크롤링

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
season_list = ['2021-22', '2020-21', '2019-20', '2018-19', '2017-18', '2016-17', '2015-16', '2014-15']
best5_list_21 = []
best5_position_21 = []
driver.get(f'https://namu.wiki/w/%ED%8B%80:KBL%20%EB%B2%A0%EC%8A%A4%ED%8A%B8%205({season_list[0]}%20%EC%8B%9C%EC%A6%8C)')

driver.find_element(By.XPATH, '//*[@id="lYnqWtHhT"]/div[2]/div/div/div/div/div[1]/div[6]/div/div/div/div/div/div[6]/div/div/div/div/div/div/div/div/div[15]/div[1]/div/div[1]/table/tbody/tr[2]/td/div/dl/dt').click()
time.sleep(1)

for i in range(1,6):
    name = driver.find_element(By.XPATH, f'//*[@id="lYnqWtHhT"]/div[2]/div/div/div/div/div[1]/div[6]/div/div/div/div/div/div[6]/div/div/div/div/div/div/div/div/div[15]/div[1]/div/div[1]/table/tbody/tr[2]/td/div/dl/dd/div/div/table/tbody/tr[2]/td[{i}]/div/a').text
    position = driver.find_element(By.XPATH, f'//*[@id="lYnqWtHhT"]/div[2]/div/div/div/div/div[1]/div[6]/div/div/div/div/div/div[6]/div/div/div/div/div/div/div/div/div[15]/div[1]/div/div[1]/table/tbody/tr[2]/td/div/dl/dd/div/div/table/tbody/tr[2]/td[{i}]/div/sub/a[2]').text
    best5_list_21.append(name)
    best5_position_21.append(position)

driver.close()

In [14]:
# 2020시즌 best선수 이름 크롤링

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
season_list = ['2021-22', '2020-21', '2019-20', '2018-19', '2017-18', '2016-17', '2015-16', '2014-15']
best5_list_20 = []
best5_position_20 = []
driver.get(f'https://namu.wiki/w/%ED%8B%80:KBL%20%EB%B2%A0%EC%8A%A4%ED%8A%B8%205({season_list[2]}%20%EC%8B%9C%EC%A6%8C)')

driver.find_element(By.XPATH, '//*[@id="lYnqWtHhT"]/div[2]/div/div/div/div/div[1]/div[6]/div/div/div/div/div/div[6]/div/div/div/div/div/div/div/div/div[15]/div[1]/div/div[1]/table/tbody/tr[2]/td/div/dl/dt').click()
time.sleep(1)

for i in range(1,6):
    name = driver.find_element(By.XPATH, f'//*[@id="lYnqWtHhT"]/div[2]/div/div/div/div/div[1]/div[6]/div/div/div/div/div/div[6]/div/div/div/div/div/div/div/div/div[15]/div[1]/div/div[1]/table/tbody/tr[2]/td/div/dl/dd/div/div/table/tbody/tr[2]/td[{i}]/div/a').text
    position = driver.find_element(By.XPATH, f'//*[@id="lYnqWtHhT"]/div[2]/div/div/div/div/div[1]/div[6]/div/div/div/div/div/div[6]/div/div/div/div/div/div/div/div/div[15]/div[1]/div/div[1]/table/tbody/tr[2]/td/div/dl/dd/div/div/table/tbody/tr[2]/td[{i}]/div/sub/a[2]').text
    best5_list_20.append(name)
    best5_position_20.append(position)

driver.close()

In [16]:
# 2019시즌 best선수 이름 크롤링

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
season_list = ['2021-22', '2020-21', '2019-20', '2018-19', '2017-18', '2016-17', '2015-16', '2014-15']
best5_list_19 = []
best5_position_19 = []
driver.get(f'https://namu.wiki/w/%ED%8B%80:KBL%20%EB%B2%A0%EC%8A%A4%ED%8A%B8%205({season_list[3]}%20%EC%8B%9C%EC%A6%8C)')

driver.find_element(By.XPATH, '//*[@id="lYnqWtHhT"]/div[2]/div/div/div/div/div[1]/div[6]/div/div/div/div/div/div[6]/div/div/div/div/div/div/div/div/div[15]/div[1]/div/div[1]/table/tbody/tr[2]/td/div/dl/dt').click()
time.sleep(1)

for i in range(1,6):
    name = driver.find_element(By.XPATH, f'//*[@id="lYnqWtHhT"]/div[2]/div/div/div/div/div[1]/div[6]/div/div/div/div/div/div[6]/div/div/div/div/div/div/div/div/div[15]/div[1]/div/div[1]/table/tbody/tr[2]/td/div/dl/dd/div/div/table/tbody/tr[2]/td[{i}]/div/a').text
    position = driver.find_element(By.XPATH, f'//*[@id="lYnqWtHhT"]/div[2]/div/div/div/div/div[1]/div[6]/div/div/div/div/div/div[6]/div/div/div/div/div/div/div/div/div[15]/div[1]/div/div[1]/table/tbody/tr[2]/td/div/dl/dd/div/div/table/tbody/tr[1]/td[{i}]/div/a/span').text
    best5_list_19.append(name)
    best5_position_19.append(position)

driver.close()

In [17]:
# 2018시즌 best선수 이름 크롤링

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
season_list = ['2021-22', '2020-21', '2019-20', '2018-19', '2017-18', '2016-17', '2015-16', '2014-15']
best5_list_18 = []
best5_position_18 = []
driver.get(f'https://namu.wiki/w/%ED%8B%80:KBL%20%EB%B2%A0%EC%8A%A4%ED%8A%B8%205({season_list[4]}%20%EC%8B%9C%EC%A6%8C)')

driver.find_element(By.XPATH, '//*[@id="lYnqWtHhT"]/div[2]/div/div/div/div/div[1]/div[6]/div/div/div/div/div/div[6]/div/div/div/div/div/div/div/div/div[15]/div[1]/div/div[1]/table/tbody/tr[2]/td/div/dl/dt').click()
time.sleep(1)

for i in range(1,6):
    name = driver.find_element(By.XPATH, f'//*[@id="lYnqWtHhT"]/div[2]/div/div/div/div/div[1]/div[6]/div/div/div/div/div/div[6]/div/div/div/div/div/div/div/div/div[15]/div[1]/div/div[1]/table/tbody/tr[2]/td/div/dl/dd/div/div/table/tbody/tr[2]/td[{i}]/div/a').text
    position = driver.find_element(By.XPATH, f'//*[@id="lYnqWtHhT"]/div[2]/div/div/div/div/div[1]/div[6]/div/div/div/div/div/div[6]/div/div/div/div/div/div/div/div/div[15]/div[1]/div/div[1]/table/tbody/tr[2]/td/div/dl/dd/div/div/table/tbody/tr[1]/td[{i}]/div/a/span').text
    best5_list_18.append(name)
    best5_position_18.append(position)

driver.close()

In [18]:
# 2017시즌 best선수 이름 크롤링

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
season_list = ['2021-22', '2020-21', '2019-20', '2018-19', '2017-18', '2016-17', '2015-16', '2014-15']
best5_list_17 = []
best5_position_17 = []
driver.get(f'https://namu.wiki/w/%ED%8B%80:KBL%20%EB%B2%A0%EC%8A%A4%ED%8A%B8%205({season_list[5]}%20%EC%8B%9C%EC%A6%8C)')

driver.find_element(By.XPATH, '//*[@id="lYnqWtHhT"]/div[2]/div/div/div/div/div[1]/div[6]/div/div/div/div/div/div[6]/div/div/div/div/div/div/div/div/div[15]/div[1]/div/div[1]/table/tbody/tr[2]/td/div/dl/dt').click()
time.sleep(1)

for i in range(1,6):
    name = driver.find_element(By.XPATH, f'//*[@id="lYnqWtHhT"]/div[2]/div/div/div/div/div[1]/div[6]/div/div/div/div/div/div[6]/div/div/div/div/div/div/div/div/div[15]/div[1]/div/div[1]/table/tbody/tr[2]/td/div/dl/dd/div/div/table/tbody/tr[2]/td[{i}]/div/a').text
    position = driver.find_element(By.XPATH, f'//*[@id="lYnqWtHhT"]/div[2]/div/div/div/div/div[1]/div[6]/div/div/div/div/div/div[6]/div/div/div/div/div/div/div/div/div[15]/div[1]/div/div[1]/table/tbody/tr[2]/td/div/dl/dd/div/div/table/tbody/tr[1]/td[{i}]/div/a/span').text
    best5_list_17.append(name)
    best5_position_17.append(position)

driver.close()

In [22]:
# 2016시즌 best선수 이름 크롤링

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
season_list = ['2021-22', '2020-21', '2019-20', '2018-19', '2017-18', '2016-17', '2015-16', '2014-15']
best5_list_16 = []
best5_position_16 = []
driver.get(f'https://namu.wiki/w/%ED%8B%80:KBL%20%EB%B2%A0%EC%8A%A4%ED%8A%B8%205({season_list[6]}%20%EC%8B%9C%EC%A6%8C)')


driver.find_element(By.XPATH, '//*[@id="lYnqWtHhT"]/div[2]/div/div/div/div/div[1]/div[6]/div/div/div/div/div/div[6]/div/div/div/div/div/div/div/div/div[15]/div[1]/div/div[1]/table/tbody/tr[2]/td/div/dl/dt').click()
time.sleep(1)

for i in range(1,6):
    name = driver.find_element(By.XPATH, f'//*[@id="lYnqWtHhT"]/div[2]/div/div/div/div/div[1]/div[6]/div/div/div/div/div/div[6]/div/div/div/div/div/div/div/div/div[15]/div[1]/div/div[1]/table/tbody/tr[2]/td/div/dl/dd/div/div/table/tbody/tr[2]/td[{i}]/div/a').text
    position = driver.find_element(By.XPATH, f'//*[@id="lYnqWtHhT"]/div[2]/div/div/div/div/div[1]/div[6]/div/div/div/div/div/div[6]/div/div/div/div/div/div/div/div/div[15]/div[1]/div/div[1]/table/tbody/tr[2]/td/div/dl/dd/div/div/table/tbody/tr[1]/td[{i}]/div/a/span').text
    best5_list_16.append(name)
    best5_position_16.append(position)
    
driver.close()

In [19]:
# 2015시즌 best선수 이름 크롤링

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
season_list = ['2021-22', '2020-21', '2019-20', '2018-19', '2017-18', '2016-17', '2015-16', '2014-15']
best5_list_15 = []
best5_position_15 = []
driver.get(f'https://namu.wiki/w/%ED%8B%80:KBL%20%EB%B2%A0%EC%8A%A4%ED%8A%B8%205({season_list[7]}%20%EC%8B%9C%EC%A6%8C)')


driver.find_element(By.XPATH, '//*[@id="lYnqWtHhT"]/div[2]/div/div/div/div/div[1]/div[6]/div/div/div/div/div/div[6]/div/div/div/div/div/div/div/div/div[15]/div[1]/div/div[1]/table/tbody/tr[2]/td/div/dl/dt').click()
time.sleep(1)

for i in range(1,6):
    name = driver.find_element(By.XPATH, f'//*[@id="lYnqWtHhT"]/div[2]/div/div/div/div/div[1]/div[6]/div/div/div/div/div/div[6]/div/div/div/div/div/div/div/div/div[15]/div[1]/div/div[1]/table/tbody/tr[2]/td/div/dl/dd/div/div/table/tbody/tr[2]/td[{i}]/div/a').text
    position = driver.find_element(By.XPATH, f'//*[@id="lYnqWtHhT"]/div[2]/div/div/div/div/div[1]/div[6]/div/div/div/div/div/div[6]/div/div/div/div/div/div/div/div/div[15]/div[1]/div/div[1]/table/tbody/tr[2]/td/div/dl/dd/div/div/table/tbody/tr[1]/td[{i}]/div/a/span').text
    best5_list_15.append(name)
    best5_position_15.append(position)
    
driver.close()

In [34]:
season = ['22', '21', '20', '19', '18', '17', '16', '15']
best5_list_result=[]
best5_position_result=[]

best5_list_result.append(best5_list_22)
best5_position_result.append(best5_position_22)
best5_list_result.append(best5_list_21)
best5_position_result.append(best5_position_21)
best5_list_result.append(best5_list_20)
best5_position_result.append(best5_position_20)
best5_list_result.append(best5_list_19)
best5_position_result.append(best5_position_19)
best5_list_result.append(best5_list_18)
best5_position_result.append(best5_position_18)
best5_list_result.append(best5_list_17)
best5_position_result.append(best5_position_17)
best5_list_result.append(best5_list_16)
best5_position_result.append(best5_position_16)
best5_list_result.append(best5_list_15)
best5_position_result.append(best5_position_15)

season_best5_list_result = sum(best5_list_result, [])
season_best5_position_result = sum(best5_position_result, [])

In [36]:
season_best5_df = pd.DataFrame(season_best5_list_result, columns = ['player'])
season_best5_df['position'] = season_best5_position_result

best_df = pd.merge(season_best5_df, all_player_df, how = 'outer', on='player')
best5_df = best_df.dropna()
best5_df.to_csv("best5.csv", index = False)

In [37]:
#best5 선수 기록 크롤링

best5_list = best5_df['player'].to_list()
driver = webdriver.Chrome(r'C:/Users/Yun/Desktop/Jupyter/chromedriver.exe')
G_r=[]  
PPG_r=[]   
FG_r=[]   
P3_r=[] 
FT_r=[]  
RPG_r=[]    
APG_r=[]   
SPG_r=[]   
BPG_r=[]   
TO_r=[]   
PF_r=[]   
EFF_r=[]  

for best5_player in best5_list:
    G=[]    #게임수
    PPG=[]    #경기당 평균 득점
    FG=[]    #필드골 성공
    P3=[]    #3점슛 성공
    FT=[]    #자유투 성공
    RPG=[]    #경기당 리바운드
    APG=[]    #경기당 어시스트
    SPG=[]    #경기당 스틸
    BPG=[]    #경기당 블록
    TO=[]    #턴오버
    PF=[]    #파울
    EFF=[]    #효율성수치
    driver.get('https://kbl.or.kr/player-staff/player')
    search_space = driver.find_element(By.CLASS_NAME, "cs")
    search_space.clear()
    search_space.send_keys(best5_player)
    driver.find_element(By.XPATH, '//*[@id="container"]/div/div[3]/div/button/i').click()
    firstname = driver.find_element(By.CSS_SELECTOR, '#container > div > div.con-search > ul').text
    time.sleep(1)
    
    #이정현 선수는 동명이인이 있어 아래와 같이 가져왔습니다.
    if best5_player == '이정현':
        player_detail = f'//*[@id="hangul-{firstname}"]/ul/li[2]/div[1]'
        driver.find_element(By.XPATH, player_detail).click()
        time.sleep(3)
    else:
        player_detail = f'//*[@id="hangul-{firstname}"]/ul/li/div[1]'
        driver.find_element(By.XPATH, player_detail).click()
        time.sleep(3)
    
    table = driver.find_element(By.XPATH, '//*[@id="container"]/div/div[4]/div[2]/table/tbody')
    tr = table.find_elements(By.TAG_NAME, "tr")
    total_season = len(tr)
    int(total_season)
    time.sleep(1)
    
    for i in range(1,4):
        g = driver.find_element(By.XPATH, f'//*[@id="container"]/div/div[4]/div[2]/table/tbody/tr[{total_season-i}]/td[3]').text
        ppg = driver.find_element(By.XPATH, f'//*[@id="container"]/div/div[4]/div[2]/table/tbody/tr[{total_season-i}]/td[5]').text
        fg = driver.find_element(By.XPATH, f'//*[@id="container"]/div/div[4]/div[2]/table/tbody/tr[{total_season-i}]/td[6]').text
        p3 = driver.find_element(By.XPATH, f'//*[@id="container"]/div/div[4]/div[2]/table/tbody/tr[{total_season-i}]/td[7]').text
        ft = driver.find_element(By.XPATH, f'//*[@id="container"]/div/div[4]/div[2]/table/tbody/tr[{total_season-i}]/td[8]').text
        rpg = driver.find_element(By.XPATH, f'//*[@id="container"]/div/div[4]/div[2]/table/tbody/tr[{total_season-i}]/td[11]').text
        apg = driver.find_element(By.XPATH, f'//*[@id="container"]/div/div[4]/div[2]/table/tbody/tr[{total_season-i}]/td[12]').text
        spg = driver.find_element(By.XPATH, f'//*[@id="container"]/div/div[4]/div[2]/table/tbody/tr[{total_season-i}]/td[13]').text
        bpg = driver.find_element(By.XPATH, f'//*[@id="container"]/div/div[4]/div[2]/table/tbody/tr[{total_season-i}]/td[14]').text
        to = driver.find_element(By.XPATH, f'//*[@id="container"]/div/div[4]/div[2]/table/tbody/tr[{total_season-i}]/td[15]').text
        pf = driver.find_element(By.XPATH, f'//*[@id="container"]/div/div[4]/div[2]/table/tbody/tr[{total_season-i}]/td[16]').text
        eff = float(ppg)+float(rpg)+float(apg)+float(bpg)-float(to)
        
        G.append(g)
        PPG.append(ppg)
        FG.append(fg)
        P3.append(p3)
        FT.append(ft)
        RPG.append(rpg)
        APG.append(apg)
        SPG.append(spg)
        BPG.append(bpg)
        TO.append(to)
        PF.append(pf)
        EFF.append(eff)
        time.sleep(2)

    G_r.append(sum([float(item_G) for item_G in G])/3)
    PPG_r.append(sum([float(item_PPG) for item_PPG in PPG])/3)
    FG_r.append(sum([float(item_FG) for item_FG in FG])/3)
    P3_r.append(sum([float(item_P3) for item_P3 in P3])/3)
    FT_r.append(sum([float(item_FT) for item_FT in FT])/3)
    RPG_r.append(sum([float(item_RPG) for item_RPG in RPG])/3)
    APG_r.append(sum([float(item_APG) for item_APG in APG])/3)
    SPG_r.append(sum([float(item_SPG) for item_SPG in SPG])/3)
    BPG_r.append(sum([float(item_BPG) for item_BPG in BPG])/3)
    TO_r.append(sum([float(item_TO) for item_TO in TO])/3)
    PF_r.append(sum([float(item_PF) for item_PF in PF])/3)
    EFF_r.append(sum([float(item_EFF) for item_EFF in EFF])/3)

best5_rookie_df = pd.DataFrame(best5_list, columns = ['player'])
best5_rookie_df['게임수'] = G_r
best5_rookie_df['평균득점'] = PPG_r
best5_rookie_df['필드골'] = FG_r
best5_rookie_df['3점'] = P3_r
best5_rookie_df['자유투'] = FT_r
best5_rookie_df['리바운드'] = RPG_r
best5_rookie_df['어시스트'] = APG_r
best5_rookie_df['스틸'] = SPG_r
best5_rookie_df['블록'] = BPG_r
best5_rookie_df['턴오버'] = TO_r
best5_rookie_df['파울'] = PF_r
best5_rookie_df['효율성'] = EFF_r

best5_rookie_df.to_csv("best5_rookie.csv", index = False)

C:\Users\Yun\AppData\Local\Temp\ipykernel_10480\1047211636.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'C:/Users/Yun/Desktop/Jupyter/chromedriver.exe')
